In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [66]:
import pandas as pd
import numpy as np

test_seg_list = []
test_seg_list.append(0)

def div(a):
#     print("Running seg{0} ... ".format(a))
    df = pd.read_csv("./data/segData/df_seg%s.csv"%a)
    df.drop(["Unnamed: 0", "MemberId"], axis=1, inplace=True)
#     print("Sum of data(for this seg): {0} ".format(len(df)))
    msk = np.random.rand(len(df)) < 0.1
    train = df[msk]
    test = df[~msk]
    
#     print(train["SalesOrderSlaveTotalPayment"].max())
#     print(train["SalesOrderSlaveTotalPayment"].min())
    
    train["SalesOrderSlaveTotalPayment"] = norm(train["SalesOrderSlaveTotalPayment"])
#     print(train["SalesOrderSlaveTotalPayment"])
    test["SalesOrderSlaveTotalPayment"] = norm(test["SalesOrderSlaveTotalPayment"])
    
#     print("Sum of training data(for this seg): {0}".format(len(train)))
#     print("Sum of testing data(for this seg): {0}".format(len(test)))
    before = test_seg_list[-1]
    test_seg_list.append(len(test))
    
#     print("...")
    return train, test

def norm(s):
    result = (s - s.min())/(s.max()-s.min())
    return(result)

trainingData, testingData = div(2)
# df0, df1 = div(2)
# trainingData=pd.concat([trainingData,df0],ignore_index=True)  
# print(len(trainingData))
# testingData=pd.concat([testingData,df1],ignore_index=True)  
# print(len(testingData))

for i in range(3,21):
    df0, df1 = div(i)
    trainingData = pd.concat([trainingData,df0],ignore_index=True)  
#     print(len(trainingData))
    testingData = pd.concat([testingData,df1],ignore_index=True)  
#     print(len(testingData))

trainingData["num"] = norm(trainingData["num"])
testingData["num"] = norm(testingData["num"])
# print("Sum of training data: {0}".format(len(trainingData)))
# print("Sum of testing data: {0}".format(len(testingData)))
# print(test_seg_list)

/home/sopper08/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sopper08/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [67]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

def get_testing_labels(training_samples, training_labels, testing_samples):
    normalizer       = Normalizer().fit(training_samples)
    training_samples = normalizer.transform(training_samples)
    testing_samples  = normalizer.transform(testing_samples)

    pca              = PCA(n_components=0.8, whiten=True).fit(training_samples)
    training_samples = pca.transform(training_samples)
    testing_samples  = pca.transform(testing_samples)

    svm_classifier   = SVC(C=2.8).fit(training_samples, training_labels)
    return svm_classifier.predict(testing_samples)

def main():
    # Read training and testing data
    training_df  = trainingData
    testing_df   = testingData
    print("Training set has {0[0]} rows and {0[1]} columns".format(training_df.shape))
    print("Test set has {0[0]} rows and {0[1]} columns".format(testing_df.shape))

    # Get training samples and labels from training data
    training_labels     = training_df['label'].values
    training_samples    = training_df.drop(['label'], axis = 1).values
    # Get testing samples from testing data
    real_testing_label  = testing_df['label']
    testing_samples     = testing_df.drop(['label'], axis = 1).values
    testing_labels      = get_testing_labels(training_samples, training_labels, testing_samples)

    # Writing testing labels to CSV
    df_label   = pd.DataFrame(testing_labels, columns=['label'])
    df_label.index     += 1
    df_label.index.name = 'label'
    df_label["real_testing_label"] = real_testing_label
    df_label.to_csv('./data/training&testingData/testing_labels_df.csv', sep=',', index = False)
    print('testingLabel finish!')
    
    # 計算準確率
    df_label['diff'] = df_label['real_testing_label']-df_label['label']
    len_df = len(df_label)
    accuracy = (len_df - df_label['diff'].abs().sum())/len_df*100
    print("all Accuracy: {0}%".format(accuracy))
    
    index0 = test_seg_list[0]
    index1 = test_seg_list[0]-1
    for i in range(0,19):
        index2 = test_seg_list[i+1]
        N = index2 + 1
        index0 = index1 + 1
        index1 = index1 + index2
#         print(i, index2)
#         print(index0, index1)
        
        
        accuracy = (N - df_label.iloc[index0:index1,-1].abs().sum())/N*100
        print("num = {0} Accuracy: {1}%".format(i+2, accuracy))

if __name__ == "__main__":
    main()

Training set has 75446 rows and 40 columns
Test set has 677004 rows and 40 columns
testingLabel finish!
all Accuracy: 65.92619836810417%
num = 2 Accuracy: 56.67292257662769%
num = 3 Accuracy: 63.20148246436824%
num = 4 Accuracy: 67.57175639469922%
num = 5 Accuracy: 70.76342504827879%
num = 6 Accuracy: 72.72524199553239%
num = 7 Accuracy: 75.08306774259171%
num = 8 Accuracy: 76.60858210635328%
num = 9 Accuracy: 77.82800117405341%
num = 10 Accuracy: 79.47238252267105%
num = 11 Accuracy: 80.44729640464283%
num = 12 Accuracy: 82.13367609254499%
num = 13 Accuracy: 82.28402032749858%
num = 14 Accuracy: 84.00137033230558%
num = 15 Accuracy: 83.8835349600164%
num = 16 Accuracy: 85.44748969197187%
num = 17 Accuracy: 85.73839662447259%
num = 18 Accuracy: 86.95795006570303%
num = 19 Accuracy: 87.0045731707317%
num = 20 Accuracy: 88.32244008714596%
